In [1]:
#importing dependencies

## need to install yfinance before using it.
# > pip install yfinance

import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Creating a dictionary of major world indices with Ticker, Index name and Country Code

Country_indices = {"Ticker":["^GSPC","^FTSE","^GDAXI","^N225","^FCHI","^HSI","HXT.TO","^AXJO"],
                   "Index_name":["S&P 500","FTSE 100","DAX PERFORMANCE-INDEX","Nikkei 225","CAC 40","HANG SENG INDEX","Horizons S&P/TSX 60 Index","S&P/ASX 200"],
                   "Country_code":["US","GB","DE","JP","FR","CN","CA","AU"],
                  }

#converting the dictionary to a pandas DataFrame
Country_indices_df = pd.DataFrame(Country_indices)
Country_indices_df

,Ticker,Index_name,Country_code
0,^GSPC,S&P 500,US
1,^FTSE,FTSE 100,GB
2,^GDAXI,DAX PERFORMANCE-INDEX,DE
3,^N225,Nikkei 225,JP
4,^FCHI,CAC 40,FR
5,^HSI,HANG SENG INDEX,CN
6,HXT.TO,Horizons S&P/TSX 60 Index,CA
7,^AXJO,S&P/ASX 200,AU


In [3]:
#Creating an empty object to hold index prices 
df = {}

#looping through tickers , for each ticker download the prices for a start and end date range
for ticker in Country_indices_df["Ticker"]:
    df[ticker] = yf.download(ticker, start="2020-03-01", end="2020-04-22")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
#display the first ticker's data 
df["^GSPC"].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-02,2974.280029,3090.959961,2945.189941,3090.229980,3090.229980,6376400000
2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000
2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000
2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000
2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000


In [5]:
# Looping through the country indices dataframe
for index,row in Country_indices_df.iterrows():
    
    #for each ticker , add new columns Country, Ticker and Index name , populate data from the country indices dataframe
    df[row["Ticker"]]['Country'] = row["Country_code"]
    df[row["Ticker"]]['Ticker'] = row["Ticker"]
    df[row["Ticker"]]['Index_Name'] = row["Index_name"]
    
    #reset index
    df[row["Ticker"]].reset_index(inplace = True)
    
    

In [6]:
#Create a copy of the first ticker dataFrame 
US_Index_df = df["^GSPC"].copy()
US_Index_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Country,Ticker,Index_Name
0,2020-03-02,2974.280029,3090.959961,2945.189941,3090.229980,3090.229980,6376400000,US,^GSPC,S&P 500
1,2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000,US,^GSPC,S&P 500
2,2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000,US,^GSPC,S&P 500
3,2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000,US,^GSPC,S&P 500
4,2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000,US,^GSPC,S&P 500


In [7]:
#Create a copy of rest of the ticker dataFrames

GB_Index_df = df["^FTSE"].copy()
DE_Index_df = df["^GDAXI"].copy()
JP_Index_df = df["^N225"].copy()
FR_Index_df = df["^FTSE"].copy()
CN_Index_df = df["^HSI"].copy()
CA_Index_df = df["HXT.TO"].copy()
AU_Index_df = df["^AXJO"].copy()

In [8]:
# conneting to database 
connection_string = "postgres:postgres@localhost:5432/covid19_db"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# get the table name 
engine.table_names()

['us_states_cases',
 'state',
 'country',
 'index_prices',
 'country_cases',
 'us_unemployment_stats',
 'gas_price',
 'hospital_beds']

In [10]:
# loading to index_prices table. 
# since this is the first data set load , if the table exists , replace the data 

US_Index_df.to_sql(name='index_prices', con=engine, if_exists='replace', index=False)

In [11]:
# selecting data from table

pd.read_sql_query('select * from index_prices', con=engine).head()

,Date,Open,High,Low,Close,Adj Close,Volume,Country,Ticker,Index_Name
0,2020-03-02,2974.280029,3090.959961,2945.189941,3090.229980,3090.229980,6376400000,US,^GSPC,S&P 500
1,2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000,US,^GSPC,S&P 500
2,2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000,US,^GSPC,S&P 500
3,2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000,US,^GSPC,S&P 500
4,2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000,US,^GSPC,S&P 500


In [12]:
# loading to index_prices table. 
# since this is the subsequent data set loads , append the data , so we dont lose the first set of data

GB_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
DE_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
JP_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
FR_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
CN_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
CA_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)
AU_Index_df.to_sql(name='index_prices', con=engine, if_exists='append', index=False)

In [13]:
#selecting first five records from table

pd.read_sql_query('select * from index_prices', con=engine).head()

,Date,Open,High,Low,Close,Adj Close,Volume,Country,Ticker,Index_Name
0,2020-03-02,2974.280029,3090.959961,2945.189941,3090.229980,3090.229980,6376400000,US,^GSPC,S&P 500
1,2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000,US,^GSPC,S&P 500
2,2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000,US,^GSPC,S&P 500
3,2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000,US,^GSPC,S&P 500
4,2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000,US,^GSPC,S&P 500


In [14]:
#selecting last five records from table

pd.read_sql_query('select * from index_prices', con=engine).tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Country,Ticker,Index_Name
281,2020-04-16,5466.700195,5466.700195,5345.000000,5416.299805,5416.299805,1115600,AU,^AXJO,S&P/ASX 200
282,2020-04-17,5416.299805,5563.600098,5416.299805,5487.500000,5487.500000,1125300,AU,^AXJO,S&P/ASX 200
283,2020-04-20,5486.299805,5490.600098,5353.000000,5353.000000,5353.000000,855400,AU,^AXJO,S&P/ASX 200
284,2020-04-21,5353.000000,5353.000000,5214.500000,5221.299805,5221.299805,960500,AU,^AXJO,S&P/ASX 200
285,2020-04-22,5221.299805,5251.700195,5100.700195,5221.200195,5221.200195,996700,AU,^AXJO,S&P/ASX 200
